<a href="https://colab.research.google.com/github/aakanksha1111/ClusterAnalysisHerwig/blob/master/GAN_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading and picking up our training data

First, we import all the packages and libraries needed to run the model.

In [ ]:
%tensorflow_version 2.x
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy.random import rand
from numpy.random import randn
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files

Two Methods of uploading data to google colab. \\
Method 1 : Gives Public access, without access of any drive. Right click the file and cLick at get link option, then change the privacy of data file to "anyone with the link can view". Copy the document id. \\
Method 2 : Requires a drive access, one needs to upload all files in the drive and then mount the drive here.

In [ ]:
#Method 1 : Public access
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=FILE_ID' -O data2
data=np.load('/content/data2')
data.shape

In [ ]:
#Method 2 :
!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#2. Get the file
downloaded = drive.CreateFile({'id':"FILE_ID"})   # replace the id with id of file you want to access
downloaded.GetContentFile('data.npy')

data = np.load('data.npy', encoding='bytes')
data
data.shape

Now we want that each row contains varaibles coressponding to same decay event i.e. Cluster decaying to two decay products. So,we reshape the data by clubbing three rows into one. Thus, no of rows reduce to 1/3 of previous value and no of columns become 3 times. 

In [ ]:
data=data.reshape((int(data.shape[0]/3),int(data.shape[1]*3)))
data.shape

Here are the details of how data is stored after reshaping :
*   It is now 2D numpy array
*   Number of rows = 1/3 of original value
*   Number of columns = 24 (8*3)
*   Index are 0,1, 2,.....,23
*   Every row has variables correspoinding to same decay event.
*   Indexing of columns is as follows : \\
Cluster - 0 to 7, Particle1 - 8 to 15, Particle2 - 16 to 23

| Main Event Number | PDG ID| Energy | Px | Py |Pz | Invariant Mass | On Shell Condition Value|

Now we weite a code which will extract those decay events which has pi+, pi- or pi0 whose PDG IDs are 211 , -211 , 111 respectively.

In [ ]:
# data.shape[0] = 275595
i=0 
k=0
data_pi = zeros((1,24))                # Create a temporary row of zerores
while i < data.shape[0]:               # Go through each row of data
  set=[9,17]  # Index of particle IDs
  flag=0   # counter
  for j in set:
    if (data[i][j]==111):          # Increase the counter if particle is pi+ || pi- || pi0
      flag=flag+1
  if flag==2 :                                  # Separate the events where both particles are pions, counter should be 2
    t = data[i].reshape((1,24))
    data_pi = np.concatenate((data_pi,t))       # Concatenate
  i=i+1
data_pi = np.delete(data_pi,0,axis=0)        # Delete the first temporary row of zerores

Now we have managed to extract such events which have pi+, pi- or pi0 only

In [ ]:
data_pi.shape[0]           

Now we set up three counters and examine the various kinds of decay process combinations which are possible or not : \\
(Note that, the PDG IDs for decay products are at index number 9 and 17)

In [ ]:
i=0
k=0
data_pi_2 = zeros((1,24))             # Create a temporary row of zerores
data_pi_3 = zeros((1,24))             # Create a temporary row of zerores
data_pi_4 = zeros((1,24))             # Create a temporary row of zerores
data_pi_5 = zeros((1,24))             # Create a temporary row of zerores
data_pi_6 = zeros((1,24))             # Create a temporary row of zerores
data_pi_7 = zeros((1,24))             # Create a temporary row of zerores
while i < data_pi.shape[0]:
  set=[9,17]
  c1=0 # counter for pi+
  c2=0 # counter for pi-
  c3=0 # counter for pi0
  for j in set:
    if (data_pi[i][j]==211):      # increment c1 if it is pi+
      c1=c1+1
    if (data_pi[i][j]==-211):       # increment c2 if it is pi-
      c2=c2+1
    if (data_pi[i][j]==111):     # increment c3 if it is pi0
      c3=c3+1  

  if (c1==1 and c2==1):                              # Separate the decays with  pi+ and pi-
    t = data_pi[i].reshape((1,24))
    data_pi_2 = np.concatenate((data_pi_2,t))        
  elif (c3==2):                                      # Separate the decays with  2 pi0
    t = data_pi[i].reshape((1,24))
    data_pi_3 = np.concatenate((data_pi_3,t))        
  elif (c2==2):                                      # Separate the decays with  2 pi-
    t = data_pi[i].reshape((1,24))
    data_pi_4 = np.concatenate((data_pi_4,t))        
  elif (c1==2):                                      # Separate the decays with  2 pi+
    t = data_pi[i].reshape((1,24))
    data_pi_5 = np.concatenate((data_pi_5,t))        
  elif (c2==1 and c3==1):                            # Separate the decays with  pi0 and pi-
    t = data_pi[i].reshape((1,24))
    data_pi_6 = np.concatenate((data_pi_6,t))        
  elif (c3==1 and c1==1):                            # Separate the decays with  pi0 and pi+
    t = data_pi[i].reshape((1,24))
    data_pi_7 = np.concatenate((data_pi_7,t))        
  else:
    only
    k=k+1
       
  i=i+1
data_pi_2 = np.delete(data_pi_2,0,axis=0)         # Delete the first temporary rows of zerores
data_pi_3 = np.delete(data_pi_3,0,axis=0)         # Delete the first temporary rows of zerores
data_pi_4 = np.delete(data_pi_4,0,axis=0)         # Delete the first temporary rows of zerores
data_pi_5 = np.delete(data_pi_5,0,axis=0)         # Delete the first temporary rows of zerores  
data_pi_6 = np.delete(data_pi_6,0,axis=0)         # Delete the first temporary rows of zerores 
data_pi_7 = np.delete(data_pi_7,0,axis=0)         # Delete the first temporary rows of zerores

In [ ]:
print(k)
print(data_pi_2.shape)
print(data_pi_3.shape)
print(data_pi_4.shape)
print(data_pi_5.shape)
print(data_pi_6.shape)
print(data_pi_7.shape)

In the previous cell, let us examine what is the correspondance of different values : \\
data_pi_2 :  Cluster -> pi+ & pi- \\
data_pi_3 :  Cluster -> pi0 & pi0 \\
data_pi_4 :  Cluster -> pi- & pi-  (Never Happens) \\
data_pi_5 :  Cluster -> pi+ & pi+  (Never Happens) \\
data_pi_6 :  Cluster -> pi0 & pi- \\
data_pi_7 :  Cluster -> pi0 & pi+  \\


Approach 1 : \\
We will concern ourselves only with pi+ & pi- decays, then data_pi_2 will be used everywhere further and mass = 139.57 MeV (for both pi+ and pi-) will be used. \\
Approach 2 : \\
We will concern ourselves with cluster decay decaying to pi0 & pi0, then data_pi_3 will be used everywhere further and mass = 134.98 MeV will be used. \\

lets, pick approach 1 this time. \\
First of all, we will import the data of decay of both pi0 into our temporary data that we will work on.

In [ ]:
tempdata = data_pi_3 

In [ ]:
m = 134.98

# Here we begin with GAN implementation :

Following cell block generates n real samples, that is, it randomly extracts n rows from the temporary data we work on. It will extract specific variables only that we consider for analysis. \\
px , py, pz  of Cluster and a Particle are extracted, thus shape of sample will be (n,6). \\
Indexing as follows : \\
0,1,2 = px, py ,pz of Cluster \\
3,4,5 = px, py ,pz of any one particle \\

In [ ]:
def generate_real_samples(n):
  i=0
  X1 = zeros((n,3))
  X2 = zeros((n,3))
  b=tempdata[np.random.choice(tempdata.shape[0], n, replace=True), :]
  while i < (n):
    set=[3,4,5] # index for px,py,pz of cluster
    for j in set:
      X1[i][j-3]=b[i][j] # store those values in sample
      
    set=[11,12,13] # index for px,py,pz of Particle (any one)
    for j in set:
      X2[i][j-11]=b[i][j] # store those values in sample
    
    i=i+1
  X = hstack((X1, X2))
  y = ones((n, 1)) 
  return X, y

Following cell block Plots the histogram for Squared Trans. Momentum Distribution, for both Cluster & Particle # Cluster has color red and Particle has blue. It takes as input the sample generated only, ( sample can be real or fake).

In [ ]:
def display(X,Y):
  v=zeros((X.shape[0],2))
  i=0
  while i < X.shape[0]:
    v[i][0]= (X[i][0]*X[i][0])+(X[i][1]*X[i][1]) # px*px + py*py for Cluster
    v[i][1]= (X[i][3]*X[i][3])+(X[i][4]*X[i][4]) # px*px + py*py for Particle
    i=i+1
  #plt.subplot(1, 2, 1)
  #plt.hist(v[:,0],color='red',alpha=0.5,bins=np.linspace(0,1000000000*0.2*0.25*0.2,50)) # Red For Cluster
  #plt.hist(v[:,1],color='blue',alpha=0.5,bins=np.linspace(0,1000000000*0.2*0.25*0.2,50))  # Blue for Particle
  #plt.title('Squared pT Distribution of Real samples')
 

  w=zeros((Y.shape[0],2))
  i=0
  while i < Y.shape[0]:
    w[i][0]= (Y[i][0]*Y[i][0])+(Y[i][1]*Y[i][1]) # px*px + py*py for Cluster
    w[i][1]= (Y[i][3]*Y[i][3])+(Y[i][4]*Y[i][4]) # px*px + py*py for Particle
    i=i+1
  #plt.subplot(1, 2, 2)
  #plt.hist(w[:,0],color='red',alpha=0.5,bins=np.linspace(0,1000000000*0.2*0.25*0.2,50)) # Red For Cluster
  #plt.hist(w[:,1],color='blue',alpha=0.5,bins=np.linspace(0,1000000000*0.2*0.25*0.2,50))  # Blue for Particle
  #plt.title('Squared pT Distribution of Fake samples')

  g=zeros((X.shape[0],1))          # For Real Samples
  i=0
  while i < X.shape[0]:
    g[i][0]= calcmsqr(X[i][0],X[i][1],X[i][2],X[i][3],X[i][4],X[i][5],134.98,134.98)  # Calc Sq. Mass of cluster
    i=i+1
  
  h=zeros((Y.shape[0],1))         # For Fake Samples
  i=0
  while i < Y.shape[0]:
    h[i][0]= calcmsqr(Y[i][0],Y[i][1],Y[i][2],Y[i][3],Y[i][4],Y[i][5],134.98,134.98)  # Calc Sq. Mass of cluster
    i=i+1
  
 

  fig, ax = plt.subplots(nrows=2,ncols= 2,figsize=(20, 10),sharex=True, sharey=True) 
  ax[0,0].hist(v[:,0],color='red',alpha=0.5,bins=np.linspace(0,1000000000*0.2*0.25*0.2,50),label='Cluster') # Red For Cluster
  ax[0,0].hist(v[:,1],color='blue',alpha=0.5,bins=np.linspace(0,1000000000*0.2*0.25*0.2,50),label='1st Particle')  # Blue for Particle
  ax[0,0].set_title('Squared pT Distribution of Real samples',fontweight="bold",color='yellow',fontsize=20)
  ax[0,0].legend()

  ax[0,1].hist(w[:,0],color='red',alpha=0.5,bins=np.linspace(0,1000000000*0.2*0.25*0.2,50),label='Cluster') # Red For Cluster
  ax[0,1].hist(w[:,1],color='blue',alpha=0.5,bins=np.linspace(0,1000000000*0.2*0.25*0.2,50),label='1st Particle')  # Blue for Particle
  ax[0,1].set_title('Squared pT Distribution of Fake samples',fontweight="bold",color='yellow',fontsize=20)
  ax[0,1].legend()

  ax[1,0].hist(g[:,0],color='green',bins=np.linspace(0,1000000000*0.2*0.25*0.2,50),alpha=0.5,label='Real') #
  ax[1,0].hist(h[:,0],color='purple',bins=np.linspace(0,1000000000*0.2*0.25*0.2,50),alpha=0.5,label='Fake')  # 
  ax[1,0].set_title('Squared invariant mass of Cluster Distribution',fontweight="bold",color='yellow',fontsize=20)
  ax[1,0].legend() 
  
 
  ax[1,1].axis('off')
  
  #fig.subplots_adjust(hspace=1000)
  fig.tight_layout(pad =5)
  #plt.savefig('test.pdf')         #    These two lines to 
  #files.download('test.pdf')      #    Download data
  plt.show()

Following Cell Block calculates and returns the Squared Invariant Mass of Cluster. \\
Inputs are : \\
 
px,py,pz = 3 momenta of cluster \\
px1, py1, pz1 = 3 momenta of 1st particle \\
m1,m2 = Invariant masses of particles \\

In [ ]:
def calcmsqr(px,py,pz,px1,py1,pz1,m1,m2):
  t1 = (px1*px1)+(py1*py1)+(pz1*pz1)+(m1*m1) # E1*E1
  E1 = np.sqrt(t1)
  px2 = px-px1
  py2 = py-py1
  pz2 = pz-pz1
  t2 = (px2*px2)+(py2*py2)+(pz2*pz2)+(m2*m2) # E2*E2
  E2 = np.sqrt(t2)
  E = E1 + E2
  t3 = (E*E)-(px*px)-(py*py)-(pz*pz)
  return t3

Following Cell Block prints the accuracy of Discriminator model for 100 samples. It also plots the Squared pT Distribution for both real and fake samples. This function will be called later.

In [ ]:
def summarize_performance(epoch, generator, discriminator, latent_dim, n=100):
  # prepare real samples
  x_real, y_real = generate_real_samples(n)
  # evaluate discriminator on real examples
  _, acc_real = discriminator.evaluate(x_real, y_real, verbose=0)
	# prepare fake examples
  x_fake, y_fake = generate_fake_samples(generator, latent_dim, n)
  # evaluate discriminator on fake examples
  _, acc_fake = discriminator.evaluate(x_fake, y_fake, verbose=0)
  # summarize discriminator performance
  print(epoch, acc_real, acc_fake)
  display(x_real,x_fake)

Rest of the GAN Model is almost same as Tutorial : 
[How to Develop a 1D Generative Adversarial Network From Scratch in Keras
](https://machinelearningmastery.com/how-to-develop-a-generative-adversarial-network-for-a-1-dimensional-function-from-scratch-in-keras/) \\
Only modifications are :
Increasesd the number of layers in both Discriminator & Generator model
definition in proportion to number of inputs or number of outputs ( sample size ) \\
for sample size=6 , 6/2=3 , thus all layers are increased three times. \\
Increased the size of Latent Points Generated by factor of 3000 \\
latent_dim =15 ( can also be increased in proportion) \\
n_epochs = 10000 , n_batch = 1024 , n_eval = 100

Discriminator Model :
*   The define_discriminator() : function below defines and returns the discriminator model. 
*   It takes 6 inputs here which are our three momentum values of cluster and one particle. \\
*   It is a sequential model i.e a linear stack of layers.
*   model.add() : Generates fully connected layers. It take sample size as 25*3 = 75. \\
*   Here, we have used ReLU activation function which maps any negative input too. I suggest in next study, we take  [Leaky ReLU actiavtion function](https://machinelearningmastery.com/rectified-linear-activation-function-for-deep-learning-neural-networks/) which allows a small positive gradient , this may prevents gradient from dying out during trainings which could lead to better outcomes.
* It also uses He [weight initialization](https://machinelearningmastery.com/weight-initialization-for-deep-learning-neural-networks/)and it will use a sigmoid activation function which would give output in (0,1). 
*The model will minimize the binary cross entropy loss function, and [the Adam version of stochastic gradient descent](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/) will be used because it is very effective.


In [ ]:
# define the standalone discriminator model
def define_discriminator(n_inputs=6):
	model = Sequential()
	model.add(Dense(75, activation='relu', kernel_initializer='he_uniform', input_dim=n_inputs))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

The following cell prints the summary of discriminator model :

In [ ]:
# define the discriminator model
model = define_discriminator()
# summarize the model
model.summary()
# plot the model
plot_model(model, to_file='discriminator_plot.png', show_shapes=True, show_layer_names=True)


Generator Model : Now, we define the generator model. \\

The define_generator() function below defines and returns the generator model.

*   The define_generator() function : It takes laten_dim as input and it gives six outputs (which are the momentum values of cluster and particle).
*   Latent_dim : Laten variable is hidden variable. Laten space is a multi-dimensional vector space of these variables. We define the size of latent space according to our requirement. The size of the latent dimension is parameterized in case we want to play with it later.
*   model.add() :
Generates a fully connected layer. Here, we can see that it takes a value given as 45, which is sample size, taken as 15*3 (given in tutorial).
* Here, we have used ReLU activation function which maps any negative input too. I suggest in next study, we take Leaky ReLU actiavtion function which allows a small positive gradient , this may prevents gradient from dying out during trainings which could lead to better outcomes.
* It also uses He weight initialization and it will use a linear activation function. I suggest, after normalization we must try to use sigmoid activation function which would give output in (0,1). 

In [ ]:
# define the standalone generator model
def define_generator(latent_dim, n_outputs=6):
	model = Sequential()
	model.add(Dense(45, activation='relu', kernel_initializer='he_uniform', input_dim=latent_dim))
	model.add(Dense(n_outputs, activation='linear'))
	return model

The following cell prints the summary of generator model :

In [ ]:
model = define_generator(5)
# summarize the model
model.summary()
# plot the model
plot_model(model, to_file='generator_plot.png', show_shapes=True, show_layer_names=True)


Combined generator and discriminator model : (Theory)

Now, the weights and bias in the generator are updated based on the performance of the discriminator model. If the discriminator is effectively detecting the fake samples, the generator is updated more, and when the discriminator is giving confused outputs, the generator model is updated less. Thus, the generator receives as input random points in the latent space, generates samples and provide it into the discriminator model directly, which gets classified between real or fake, after this the output of this model is used to update the the generator.  \\
The discriminator model can be trained in a standalone manner on each example as it is just a classifier. \\
The generator model is only concerned with the discriminator’s performance on fake examples. It tries to generate such samples with which it can fool the discriminator better. \\

thus, it can be seen in code below, that  we have marked the discriminator as not trainable when it is part of the GAN model so that, they can not be updated and overtrained on fake examples. \\
Similarly, when training the generator,  we want the discriminator to think that the samples output by the generator are real, not fake. Thus, when the generator is trained as part of the GAN model, rhe generated samples are marked as real (1). The discriminator will then classify the generated samples as not real (0) or a low probability value (0.3 or 0.5). \\
The backpropagation process used to update the model weights will see this as a large error and will update the weights and bias of generator model weights to correct for this error, in turn making the generator better.



Combined generator and discriminator model which will update the generator :

* Inputs: Point in latent space. \\
* Outputs: Tell if the sample is real or fake. \\
* The define_gan() function takes as arguments the already-defined generator and discriminator models and creates the new logical third model subsuming these two models. \\
* The GAN model then uses the same binary cross entropy loss function as the discriminator and the efficient Adam version of stochastic gradient descent. \\
Note : The weights in the discriminator are marked as not trainable, which only affects the weights as seen by the GAN model and not the standalone discriminator model.

In [ ]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
	# make weights in the discriminator not trainable
	discriminator.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(generator)
	# add the discriminator
	model.add(discriminator)
	# compile model
	model.compile(loss='binary_crossentropy', optimizer='adam')
	return model

Generating fake samples :

Now, we will use the generator model to generate the same number of fake samples.

the generate_latent_points() function developed in the generator section above : generates the same number of points in the latent space. 
These will then be passed to the generator model and used in training.

The generate_fake_samples() function : generates these fake samples and the associated class label of 0 which will be useful later.



In [ ]:
def generate_latent_points(latent_dim, n):
	# generate points in the latent space
	x_input = randn(latent_dim * n)*3000 
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n, latent_dim)
	return x_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n):
  # generate points in latent space
  x_input = generate_latent_points(latent_dim, n)
  # predict outputs
  X = generator.predict(x_input)
  # create class labels
  y = zeros((n, 1))
  return X, y

Training the GAN Model :
The Discriminator is trained to assign 0 to the fake
data and 1 to real data. \\
The Generator is trained such that the Discriminator assigns 1 to the fake examples produced by it. \\

we have to first update the discriminator model with real and fake samples, then update the generator via the composite model. Thus, combining elements from the train_discriminator() function and the train_gan() function. We need to make sure that the generate_fake_samples() function use the generator model to generate fake samples instead of generating random numbers, which we make sure by past g_model, latent_dim and half_batch as arguments in generate_fake_samples() function.

Here, we are manually enumerating the training epochs and for each epoch generating a half batch of real examples and a half batch of fake examples, and updating the model on each, e.g. one whole batch of examples. 

The model can then be evaluated on the generated examples and we can report the classification accuracy on the real and fake samples.

The train() function below implements this, training the model 10000 times (n_epochs)  for 1024 samples (n_batch) at an interval of 100 (n_eval).
Half batch is trained for real samples and other half for fake.  


In [ ]:
def train(g_model, d_model, gan_model, latent_dim, n_epochs=10000, n_batch=1024, n_eval=100):
	# determine half the size of one batch, for updating the discriminator
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
		# prepare real samples
		x_real, y_real = generate_real_samples(half_batch)
		# prepare fake examples
		x_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
		# update discriminator
		d_model.train_on_batch(x_real, y_real)
		d_model.train_on_batch(x_fake, y_fake)
		# prepare points in latent space as input for the generator
		x_gan = generate_latent_points(latent_dim, n_batch)
		# create inverted labels for the fake samples
		y_gan = ones((n_batch, 1))
		# update the generator via the discriminator's error
		gan_model.train_on_batch(x_gan, y_gan)
		# evaluate the model every n_eval epochs
		if (i+1) % n_eval == 0:
			summarize_performance(i, g_model, d_model, latent_dim)

Final model :
Latent dimension is made three times (5*3)
In the given model we have six inputs, When there are 2 inputs, the latent dimension was taken as 6, so here in our model we have made such changes.
Now here, we call back our previous functions and run the training in final line.

In [ ]:
# size of the latent space
latent_dim = 15
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# train model
train(generator, discriminator, gan_model, latent_dim)

This one uses the summarize performance function described above. (n=6000 is the number of iterations, though I am not sure)

In [ ]:
summarize_performance(1,generator, discriminator, latent_dim, n=6000)

The following code displays the fake samples generated by generator.

In [ ]:
x,y =generate_fake_samples(generator,latent_dim,5)
x

The following code displays the real samples.

In [ ]:
x,y =generate_real_samples(5)
x

# References :

Article : [How to Develop a 1D Generative Adversarial Network From Scratch in Keras](https://machinelearningmastery.com/how-to-develop-a-generative-adversarial-network-for-a-1-dimensional-function-from-scratch-in-keras/)

Book name :
[GANs in Action, Chapter 3 : Your first GAN: Generating handwritten digits](https://www.manning.com/books/gans-in-action)